<a href="https://colab.research.google.com/github/AleksaSubaranovic/AdaptiveAttackOnQuantumMoney/blob/main/AdaptiveAttackOnQuantumMoneyShowcase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wiesner's quantum money scheme

https://en.wikipedia.org/wiki/Quantum_money

# Imports

In [ ]:
!pip install qiskit
!pip install qiskit-ibm-runtime
!pip install qiskit[visualization]
!pip install qiskit-ibmq-provider
!pip install qiskit-aer

In [ ]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Options
import math

# Code

Generate Quantum States used in Wiesner's quantum money

In [ ]:
# def make_zero(circuit, ind):
#  do_nothing

def make_one(circuit, ind):
  circuit.x(ind)
def make_plus(circuit, ind):
  circuit.h(ind)
def make_minus(circuit, ind):
  circuit.h(ind)
  circuit.z(ind)

Quantum Random Number Generator.
- Serial Number will be unique 4 digit number
- Qubit State will be one of 4 states {0,1,+,-} and saved in memory as 2 bits

In [ ]:
class QuantumNumberGenerator:
    def __init__(self, num_bits=1):
        self.num_bits = num_bits
        self.quantum_circuit = QuantumCircuit(num_bits, num_bits)

    def generate_quantum_number(self):
        # Apply Hadamard gates to create superposition
        for qubit in range(self.num_bits):
            self.quantum_circuit.h(qubit)

        # Measure qubits to collapse superposition
        self.quantum_circuit.measure(range(self.num_bits), range(self.num_bits))

        # Simulate quantum circuit
        simulator = Aer.get_backend('qasm_simulator')
        transpiled_circuit = transpile(self.quantum_circuit, simulator)
        qobj = assemble(transpiled_circuit)
        result = simulator.run(qobj).result()

        # Extract measurement result as an integer
        counts = result.get_counts()
        quantum_number = int(list(counts.keys())[0], 2)

        return quantum_number

def Generate4DigitNumber():
  q = QuantumNumberGenerator(14)
  res = q.generate_quantum_number() % 10000
  return res

# 2 bits
def GenerateQubitValue():
  q = QuantumNumberGenerator(2)
  res = q.generate_quantum_number()
  return res

Verify Functions

In [ ]:
# does not check for same base
def Verify50(basis_bit, bit0, bit1):
  circuit = QuantumCircuit(1)

  if bit0 and bit1:
    make_minus(circuit, 0)
  elif bit0:
    make_plus(circuit, 0)
  elif bit1:
    make_one(circuit, 0)

  # for measuring
  if basis_bit:
    circuit.h(0)

  circuit.measure_all()
  simulator = Aer.get_backend('qasm_simulator')
  result = simulator.run(circuit).result()
  return int(result.get_counts().most_frequent())

# checks for base
def Verify100(basis_bit, expected_result, bit0, bit1):
  if basis_bit == bit0:
    ver = Verify50(basis_bit, bit0, bit1)
    return ver == expected_result
  else:
    return False

def ToBits(number):
  if number == 0:
    return 0, 0
  elif number == 1:
    return 0, 1
  elif number == 2:
    return 1, 0
  elif number == 3:
    return 1, 1
  return -1, -1

## Bank class

In [ ]:
class Bank:
    def __init__(self, num_cubits):
        self.quantum_money_database = dict()
        self.num_cubits = num_cubits

    # simple way to generate money
    def create_quantum_money(self):
      if len(self.quantum_money_database.keys()) >= 10000:
        return -1, -1
      s = Generate4DigitNumber()
      while s in self.quantum_money_database.keys():
        s = (s + 1) % 10000

      res = []
      for i in range(0, self.num_cubits):
        cubit_value = GenerateQubitValue()
        res.append(cubit_value)
      self.quantum_money_database[s] = res
      return s, res

    def give_quantum_money(self, recipient):
      s, q = self.create_quantum_money()
      if s == -1:
        return False
      else:
        recipient.money_arr.append((s, q))
        return True

    def verify_quantum_money(self, s, q):
      if s in self.quantum_money_database.keys():
        res = self.quantum_money_database[s]
        for i in range(0, len(q)):
          bit0, bit1 = ToBits(q[i]) # potentially forged qubit
          bit2, bit3 = ToBits(res[i]) # correct qubit
          # bit2 = base, bit3 = measured result, bit0, bit1 represent qubit that will be measured
          if (Verify100(bit2, bit3, bit0, bit1) == False):
            return False
        return True
      return False

## User class

In [ ]:
class User:
  def __init__(self):
    self.money_arr = []

  def request_money(self, emitent):
    return emitent.give_quantum_money(self)

  def verify_user_money(self, emitent, money_secret_number):
    for (s, q) in self.money_arr:
      if s == money_secret_number:
        return emitent.verify_quantum_money(s, q)

Example of workflow

In [ ]:
bank = Bank(1)
user1 = User()
user2 = User()

bank.give_quantum_money(user1)
bank.give_quantum_money(user2)

print(f"Verify User 1 {user1.verify_user_money(bank, user1.money_arr[0][0])}\n")
secret_key, qubits = user2.money_arr[0]

print("Try random guess")
for i in range(0, 4):
  print(f"-{i + 1}-")
  print(bank.verify_quantum_money(secret_key, [i]))

print("Show Bank Data")
print(bank.quantum_money_database)

Verify User 1 True

Try random guess
-1-
False
-2-
False
-3-
False
-4-
True
Show Bank Data
{3953: [3], 3414: [3]}


<ipython-input-5-06065d24e4f7>:18: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  result = simulator.run(qobj).result()


## Forge Money using Bomb Testing Attack (Adaptive attack)
- https://arxiv.org/abs/1404.1507
- Change is made to angle delta = PI / (2 * N) => PI / (N)


In [ ]:
# Example of N = 4
def drawBombTest():
  angle = math.pi / (4 * 1)
  circuit = QuantumCircuit(4 + 1)
  counter = angle
  for i in range(0, 4):
    ind = i + 1
    circuit.rx(counter, 0)

    circuit.cx(0, ind)

  circuit.measure_all()
  print(circuit)

drawBombTest()

        ┌─────────┐     ┌─────────┐     ┌─────────┐     ┌─────────┐      ░ ┌─┐»
   q_0: ┤ Rx(π/4) ├──■──┤ Rx(π/4) ├──■──┤ Rx(π/4) ├──■──┤ Rx(π/4) ├──■───░─┤M├»
        └─────────┘┌─┴─┐└─────────┘  │  └─────────┘  │  └─────────┘  │   ░ └╥┘»
   q_1: ───────────┤ X ├─────────────┼───────────────┼───────────────┼───░──╫─»
                   └───┘           ┌─┴─┐             │               │   ░  ║ »
   q_2: ───────────────────────────┤ X ├─────────────┼───────────────┼───░──╫─»
                                   └───┘           ┌─┴─┐             │   ░  ║ »
   q_3: ───────────────────────────────────────────┤ X ├─────────────┼───░──╫─»
                                                   └───┘           ┌─┴─┐ ░  ║ »
   q_4: ───────────────────────────────────────────────────────────┤ X ├─░──╫─»
                                                                   └───┘ ░  ║ »
meas: 5/════════════════════════════════════════════════════════════════════╩═»
                                        

- Verification is skipped here

In [ ]:
from qiskit.visualization import circuit_drawer
import math
from qiskit.extensions import UnitaryGate
import numpy as np

ZERO = 0
ONE = 1
PLUS = 2
MINUS = 3


c_minus_x = np.array([         [1, 0, 0, 0],
                               [0, 1, 0, 0],
                               [0, 0, 0, -1],
                               [0, 0, -1, 0]])

c_minus_x_gate = UnitaryGate(c_minus_x, label='C-X')

#
# Bomb Testing Attack
#

def BT_PLUS(q, N):
  angle = math.pi / (N)
  circuit = QuantumCircuit(1 + N, 1)
  for i in range(0, N):
    ind = i + 1
    circuit.rx(angle, 0)

    if q == 1:
      make_one(circuit, ind)
    elif q == 2:
      make_plus(circuit, ind)
    elif q== 3:
      make_minus(circuit, ind)

    circuit.cx(0, ind)
    #
    # Verify Step => shoul always pass for lagre N
    #

  circuit.measure(0, 0)
  simulator = Aer.get_backend('qasm_simulator')
  result = simulator.run(circuit).result()
  return result.get_counts().most_frequent()

def BT_MINUS(q, N):
  angle = math.pi / (N)
  circuit = QuantumCircuit(1 + N, 1)
  for i in range(0, N):
    ind = i + 1
    circuit.rx(angle, 0)

    if q == 1:
      make_one(circuit, ind)
    elif q == 2:
      make_plus(circuit, ind)
    elif q== 3:
      make_minus(circuit, ind)

    circuit.append(c_minus_x_gate, [ind, 0])
    #
    # Verify Step => shoul always pass for lagre N
    #

  circuit.measure(0, 0)
  simulator = Aer.get_backend('qasm_simulator')
  result = simulator.run(circuit).result()
  #print(circuit)
  return result.get_counts().most_frequent()

def BT_ZERO_ONE(q):
  circuit = QuantumCircuit(1, 1)
  if q == 1:
      circuit.x(0)
  circuit.measure(0, 0)
  simulator = Aer.get_backend('qasm_simulator')
  result = simulator.run(circuit).result()
  return result.get_counts().most_frequent()

def BT_FULL(q, N):
  if BT_PLUS(q, N) == "1":
    return "+"
  elif BT_MINUS(q, N) == "1":
    return "-"
  else:
    return BT_ZERO_ONE(q)

qubit1 = PLUS
plus_odds = BT_PLUS(qubit1, 20)
print(f"Results for plus {plus_odds}")
print("----")

qubit1 = MINUS
plus_odds = BT_PLUS(qubit1, 20)
print(f"Results for minus being plus {plus_odds}")
minus_odds = BT_MINUS(qubit1, 20)
print(f"Results for minus {minus_odds}")
print("----")

qubit1 = ZERO
print(f"Qubit ZERO is => {BT_FULL(qubit1, 20)}")

qubit2 = ONE
print(f"Qubit ONE => {BT_FULL(qubit2, 20)}")

qubit3 = PLUS
print(f"Qubit PLUS => {BT_FULL(qubit3, 20)}")

qubit4 = MINUS
print(f"Qubit MINUS => {BT_FULL(qubit4, 20)}")

Results for plus 1
----
Results for minus being plus 0
Results for minus 1
----
Qubit ZERO is => 0
Qubit ONE => 1
Qubit PLUS => +
Qubit MINUS => -
